<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/time_vald.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.1.0-rc0

     |████████████████████████████████| 402.3MB 36kB/s 
     |████████████████████████████████| 3.8MB 50.6MB/s 
     |████████████████████████████████| 450kB 56.9MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.1.0rc0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [2]:
pip install keras==2.3.1

     |████████████████████████████████| 378kB 2.8MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [3]:
!kaggle competitions download -c ieee-fraud-detection

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [8]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 76.4MB/s]
 84% 49.0M/58.3M [00:00<00:00, 68.5MB/s]
100% 58.3M/58.3M [00:00<00:00, 108MB/s] 
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 220MB/s]
 96% 50.0M/52.2M [00:00<00:00, 90.2MB/s]
100% 52.2M/52.2M [00:00<00:00, 132MB/s] 
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 108MB/s]


In [5]:
pip install category_encoders

     |████████████████████████████████| 102kB 2.5MB/s 


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [9]:
import pandas as pd
import numpy as np
#read autoenoded features
path = F"/content/gdrive/My Drive/autoenc.csv" 
ext=pd.read_csv(path,index_col=0)
ls=list(ext)
trn=pd.read_csv('train_transaction.csv.zip',usecols=['isFraud'])
ext=ext.loc[:trn.shape[0]]
ext['isFraud']=trn['isFraud']
import lightgbm as lgb
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
fnl=pd.DataFrame()
cols=[]
val=[]
trn.shape[0]


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


590540

In [0]:
ext=ext.dropna()
ext=ext.replace([np.inf,-np.inf],0)

In [16]:
trn=[]

#fit model with n rows and predict next 40000 rows for each column

for col in tqdm(ls):
    for i in range(50000,590540,40000):
        fit=ext.loc[0:i]
        pdct=ext.loc[i:i+40000]
        mod=lgb.LGBMClassifier(max_bins= 37,max_depth= 40,min_data_in_leaf= 110,min_gain_to_split= 1,num_leaves=48)
        mod.fit(fit[col].values.reshape(-1,1),fit['isFraud'])
        val.append(roc_auc_score(pdct['isFraud'],mod.predict_proba(pdct[col].values.reshape(-1,1))[:,1]))
        trn.append(roc_auc_score(fit['isFraud'],mod.predict_proba(fit[col].values.reshape(-1,1))[:,1]))
        cols.append(col)


100%|██████████| 252/252 [46:55<00:00, 11.17s/it]


In [0]:
ext.shape

(20000, 253)

In [0]:

#calculate mean over all time periods

fnl['cols']=cols
fnl['validation']=val
fnl['train']=trn
fnl['validation']=fnl.groupby(['cols'])['validation'].transform('mean')
fnl['train']=fnl.groupby(['cols'])['train'].transform('mean')
fnl=fnl[['validation','train','cols']]
fnl=fnl.drop_duplicates(keep='last')

In [0]:

#top 100 values for validation

fnl=fnl.sort_values(by='validation')
fnl.tail(100)['cols'].values

array(['Hidden_7_86', 'Hidden_3_39', 'Hidden_7_67', 'Hidden_7_8',
       'Hidden_4_20', 'Hidden_6_16', 'Hidden_7_70', 'Hidden_7_82',
       'Hidden_3_54', 'Hidden_7_12', 'Hidden_7_25', 'Hidden_7_36',
       'Hidden_3_84', 'Hidden_7_84', 'Hidden_3_48', 'Hidden_7_79',
       'Hidden_3_89', 'Hidden_7_57', 'Hidden_7_89', 'Hidden_6_17',
       'Hidden_3_5', 'Hidden_7_38', 'Hidden_3_62', 'Hidden_3_27',
       'Hidden_3_53', 'Hidden_3_31', 'Hidden_6_2', 'Hidden_7_66',
       'Hidden_4_17', 'Hidden_7_6', 'Hidden_7_27', 'Hidden_3_12',
       'Hidden_3_30', 'Hidden_3_81', 'Hidden_4_10', 'Hidden_7_30',
       'Hidden_7_32', 'Hidden_3_73', 'Hidden_7_45', 'Hidden_3_3',
       'Hidden_7_0', 'Hidden_7_14', 'Hidden_7_40', 'Hidden_6_15',
       'Hidden_4_23', 'Hidden_3_50', 'Hidden_7_55', 'Hidden_3_18',
       'Hidden_3_58', 'Hidden_3_33', 'Hidden_7_2', 'Hidden_4_5',
       'Hidden_3_82', 'Hidden_7_23', 'Hidden_7_72', 'Hidden_3_13',
       'Hidden_3_45', 'Hidden_7_88', 'Hidden_3_99', 'Hidden_3_56',
   